This code works through the GEDI subsetter version 0.6.0. The first part of the code works through putting in a job request for multiple AOIs at once. The second part works through a job request for a single AOI

In [1]:
#Open the MAAP host
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')


In [2]:
#Import numpy for making column inputs easier
import numpy as np

In [3]:
#Create empty variables to fill with all 29 appearances of cover_z* and pai_z* 
variables = []
for n in np.arange(0, 30,1):
    variables.append('cover_z' + str(n))
    variables.append('pai_z' + str(n))


For several AOI's run this code:

In [51]:
list_aois = [#"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Benin_EEZ.geojson",
             "https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Cameroon_EEZ.geojson",
             #"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Congo_EEZ.geojson",
             #"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/DCR_EEZ.geojson",
             #"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/EquatorialGuinea_EEZ.geojson",
             "https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Gabon_EEZ.geojson",
             "https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Gambia_EEZ.geojson",
             "https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Ghana_EEZ.geojson",
             "https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Guinea_EEZ.geojson",
             #"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Guinea-Bissau_EEZ.geojson",
             #"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/IvoryCoast_EEZ.geojson",
             #"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Liberia_EEZ.geojson",
             "https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Mauritania_EEZ.geojson",
             "https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Nigeria_EEZ.geojson",
             #"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Senegal_EEZ.geojson",
             #"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/SierraLeone_EEZ.geojson",
             #"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Togo_EEZ.geojson"
            ]

In [52]:
#Set up job results to print output during run

import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [53]:
#Set up input dictionary and import backoff to display run results
#################################

for aoi in list_aois: 
    inputs = dict(
       aoi=aoi,
       doi="L2B",
       lat="geolocation/lat_lowestmode",
       lon="geolocation/lon_lowestmode",
       beams="all",
       columns="rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
       query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
       limit = 10_000,
       temporal="-",
       output="-"
    )
    result = maap.submitJob(
        identifier="gedi-subset",
        algo_id="gedi-subset_ubuntu",
        version="0.6.0",
        queue="maap-dps-worker-32gb",
        username="abarenblitt",
    **inputs
    )
    
    job_id = result["job_id"]
    print(job_id)
    
try:
    import backoff
except:
    job_status = job_status_for(job_id)
else:
    # Check job status every 2 minutes
    @backoff.on_predicate(
        backoff.constant,
        lambda status: status not in ["Deleted", "Succeeded", "Failed"],
        interval=120,
    )
    def wait_for_job(job_id: str) -> str:
        return job_status_for(job_id)

    job_status = wait_for_job(job_id)
    job_status

bfe7fbc7-439d-4bf7-9d45-4fad5db027d4
f9dcbee4-ceff-4c3a-8d71-f7b178e5a1ef
40f361b4-4dc4-458b-aaca-b301ee5da25a
134852fd-2ab6-4191-bfca-10d6db170062
dce12a88-612a-4ebc-b937-9553ad8f2b70
5987f24d-6922-440b-99b6-0c8f49ad8949
a70a0811-3f85-40e8-8343-1f54cf2d1a92
7b5dc25d-7c05-422d-8b42-de121b0e3234
ac0e44b2-8797-4e01-b09a-ceea2c16639a


INFO:backoff:Backing off wait_for_job(...) for 94.3s (Accepted)


86278ccc-a0ad-4710-a128-cf9a245e2dd6
c0535b6c-bf0a-4879-baba-c00a255e1438


INFO:backoff:Backing off wait_for_job(...) for 42.1s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 85.1s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 35.7s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 49.8s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 104.8s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 64.6s (Running)
INFO:backoff:Backing off wait_for_job(...) for 21.0s (Running)
INFO:backoff:Backing off wait_for_job(...) for 46.3s (Running)
INFO:backoff:Backing off wait_for_job(...) for 7.8s (Running)
INFO:backoff:Backing off wait_for_job(...) for 49.5s (Running)
INFO:backoff:Backing off wait_for_job(...) for 85.2s (Running)


For a single AOI, use this code:

In [34]:
#Set up input dictionary
####################

inputs = dict(
   aoi=aoi,
   doi="L2B",
   lat="geolocation/lat_lowestmode",
   lon="geolocation/lon_lowestmode",
   beams="all",
   columns="rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
   query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
   limit = 10_000,
   temporal="-",
   output="-"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset_ubuntu",
    version="0.6.0",
    queue="maap-dps-worker-32gb",
    username="abarenblitt",
    **inputs
)
inputs

{'aoi': 'https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/Togo_EEZ.geojson',
 'doi': 'L2B',
 'lat': 'geolocation/lat_lowestmode',
 'lon': 'geolocation/lon_lowestmode',
 'beams': 'all',
 'columns': 'rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,cover_z0,pai_z0,cover_z1,pai_z1,cover_z2,pai_z2,cover_z3,pai_z3,cover_z4,pai_z4,cover_z5,pai_z5,cover_z6,pai_z6,cover_z7,pai_z7,cover_z8,pai_z8,cover_z9,pai_z9,cover_z10,pai_z10,cover_z11,pai_z11,cover_z12,pai_z12,cover_z13,pai_z13,cover_z14,pai_z14,cover_z15,pai_z15,cover_z16,pai_z16,cover_z17,pai_z17,cover_z18,pai_z18,cover_z19,pai_z19,cover_z20,pai_z20,cover_z21,pai_z21,cover_z22,pai_z22,cover_z23,pai_z23,cover_z24,pai_z24,cover_z25,pai_z25,cover_z26,pai_z26,cover_z27,pai_z27,cover_z28,pai_z28,cover_z29,pai_z29',
 'query': 'l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95',
 'limit': 10000,
 'temporal': '-',
 'output': '-'}

In [60]:
#Print job results as the job runs

try:
    import backoff
except:
    job_status = job_status_for(job_id)
else:
    # Check job status every 2 minutes
    @backoff.on_predicate(
        backoff.constant,
        lambda status: status not in ["Deleted", "Succeeded", "Failed"],
        interval=120,
    )
    def wait_for_job(job_id: str) -> str:
        return job_status_for(job_id)

    job_status = wait_for_job(job_id)
    job_status

INFO:backoff:Backing off wait_for_job(...) for 35.8s (Running)
INFO:backoff:Backing off wait_for_job(...) for 8.8s (Running)
INFO:backoff:Backing off wait_for_job(...) for 69.4s (Running)
INFO:backoff:Backing off wait_for_job(...) for 63.7s (Running)
INFO:backoff:Backing off wait_for_job(...) for 61.9s (Running)
INFO:backoff:Backing off wait_for_job(...) for 6.9s (Running)
INFO:backoff:Backing off wait_for_job(...) for 77.6s (Running)
INFO:backoff:Backing off wait_for_job(...) for 115.3s (Running)
INFO:backoff:Backing off wait_for_job(...) for 88.2s (Running)
INFO:backoff:Backing off wait_for_job(...) for 1.2s (Running)
INFO:backoff:Backing off wait_for_job(...) for 107.9s (Running)
INFO:backoff:Backing off wait_for_job(...) for 9.6s (Running)
INFO:backoff:Backing off wait_for_job(...) for 14.8s (Running)
INFO:backoff:Backing off wait_for_job(...) for 110.8s (Running)
INFO:backoff:Backing off wait_for_job(...) for 48.1s (Running)
INFO:backoff:Backing off wait_for_job(...) for 87.1s (Ru

In [62]:
job_id = "bfe7fbc7-439d-4bf7-9d45-4fad5db027d4"
job_id

'bfe7fbc7-439d-4bf7-9d45-4fad5db027d4'

In [63]:
#Print job results once

import re

# Should evaluate to 'Accepted', 'Running', 'Succeeded', or 'Failed'
re.search(r"Status>(?P<status>.+)</wps:Status>", maap.getJobStatus(job_id
).text).group('status')


'Succeeded'

In [57]:
#Get job result full text to bug Alex with
 
maap.getJobResult(job_id).text

'<wps:Result xmlns:ows="http://www.opengis.net/ows/2.0" xmlns:schemaLocation="http://schemas.opengis.net/wps/2.0/wps.xsd" xmlns:wps="http://www.opengis.net/wps/2.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><wps:JobID>c0535b6c-bf0a-4879-baba-c00a255e1438</wps:JobID><wps:Output id="traceback"><wps:Data>+ /app/gedi-subsetter/subset.sh L2B - geolocation/lat_lowestmode geolocation/lon_lowestmode all \'rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,cover_z0,pai_z0,cover_z1,pai_z1,cover_z2,pai_z2,cover_z3,pai_z3,cover_z4,pai_z4,cover_z5,pai_z5,cover_z6,pai_z6,cover_z7,pai_z7,cover_z8,pai_z8,cover_z9,pai_z9,cover_z10,pai_z10,cover_z11,pai_z11,cover_z12,pai_z12,cover_z13,pai_z13,cover_z14,pai_z14,cover_z15,pai_z15,cover_z16,pai_z16,cover_z17,pai_z17,cover_z18,pai_z18,cover_z19,pai_z19,cover_z20,pai_z20,cover_z21,pai_z21,cover_z22,pai_z22,cover_z23,pai_z23,cover_z24,pai_z24,cover_z25,pai_z25,cover_z26,pai_z26,cover_z27,pai_z27,cover_z28,pai_z28,cover_z29,pai_z29\' \'